# Narreme Visualization

This second part of the code will focus on enphasizing the visualization and exploration of the different documents and topics we extracted from the previous notebook. By doing those we can understand the linking between each documents so that to create narrative connections between them.

This notebook is part of a master's thesis project in Digital Interaction Design at Politecnico di Milano, by Federico Denni.

In [2]:
from IPython.display import clear_output
!pip install pyvis networkx seaborn spacy
!python -m spacy download xx_sent_ud_sm
clear_output()

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy #nlp
from spacy import displacy #spacy visualizer library
import pyvis #interactive visualization
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt #plots
import seaborn as sns #make these plots nice

plt.style.use('ggplot')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/research/ready_data.csv
/kaggle/input/test-data/ready_data.csv


In [4]:
try:
    #Change this directory for other dataset
    df = pd.read_csv(r'/kaggle/input/research/ready_data.csv', encoding="utf-8")
    #eliminate compound scores
    
except FileNotFoundError:
      print("Error: file not found. Please upload the file or provide the correct path.")

In [5]:
df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000


---

Now that is loaded, lets proceed to analyze and clean the different elements

In [6]:
#let's split the docs_n in sentences
nlp = spacy.load("xx_sent_ud_sm")#translate all sentences in english before using this

# Define a function to tokenize text using spaCy
def tokenize(text):
    if isinstance(text, str):
        doc = nlp(text)
        return [token.text for token in doc]
    return []

# Define a function to split text into sentences using spaCy
def split_sentences(text):
    if isinstance(text, str):
        doc = nlp(text)
        return [sent.text for sent in doc.sents]
    return []

# Apply the sentence splitting function to the new columns
df['docs_1_sentences'] = df['docs_1'].apply(split_sentences)
df['docs_2_sentences'] = df['docs_2'].apply(split_sentences)
df['docs_3_sentences'] = df['docs_3'].apply(split_sentences)

df.to_csv(r'/kaggle/working/sentences.csv', index=False)

df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score,docs_1_sentences,docs_2_sentences,docs_3_sentences
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130,[This is for everyone Just how you didn’t expe...,"[#, 4 is definitely one of the biggest critici...","[On adding symbols to the card, I disagree tha..."
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700,"[I'm a game developer, and I realized that my ...",[How me and my friends play uno],[I like the card game of uno]
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920,[You should be careful with the symbols (Golde...,"[Thanks for this list!, Soooo many games don't...","[Thanks for this list!, Soooo many games don't..."
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430,[che bella collezione],[Che bella collezione 😍😍],[Che collezione 💪💪💪]
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100,"[​@@TheHistoryGuyChannel ​Yah, as I recall, th...","[@@alymbouras it doesn't really matter, the im...","[@@alymbouras it doesn't really matter, the im..."
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000,[It seems that every country has a card game o...,[One thing most cards are missing (which is we...,[@@StefanLopuszanski I have looked you up and ...


In [7]:
#lets now identify and extract the entities type in the sentences
nlp = spacy.load("en_core_web_sm")#translate all sentences in english before using this

df = pd.read_csv(r'/kaggle/working/sentences.csv', encoding="utf-8")
columns_to_extract = ['docs_1_sentences', 'docs_2_sentences', 'docs_3_sentences']

def get_entities_and_keywords(sentences, keywords):
    entities = []
    for sentence in sentences:
        doc = nlp(sentence)
        sentence_entities = [ent.label_ for ent in doc.ents] #if MISC, add word
        for keyword in keywords:
            if keyword in sentence:
                sentence_entities.append(keyword)
                
        entities.append(sentence_entities)
    return entities

# Apply the function to each column
df['doc_1_entity_list'] = df.apply(lambda row: get_entities_and_keywords(eval(row['docs_1_sentences']), eval(row['Keywords'])), axis=1)
df['doc_2_entity_list'] = df.apply(lambda row: get_entities_and_keywords(eval(row['docs_2_sentences']), eval(row['Keywords'])), axis=1)
df['doc_3_entity_list'] = df.apply(lambda row: get_entities_and_keywords(eval(row['docs_3_sentences']), eval(row['Keywords'])), axis=1)

df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score,docs_1_sentences,docs_2_sentences,docs_3_sentences,doc_1_entity_list,doc_2_entity_list,doc_3_entity_list
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130,['This is for everyone Just how you didn’t exp...,"['#', '4 is definitely one of the biggest crit...","['On adding symbols to the card, I disagree th...","[[PERSON], [PERSON], [], [], [TIME], [PERSON],...","[[], [CARDINAL, PERSON], [], [CARDINAL, text],...","[[symbols, text, card], [ORG, cards, text, car..."
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700,"[""I'm a game developer, and I realized that my...",['How me and my friends play uno'],['I like the card game of uno'],"[[PERSON, playing, game, games, play, watching...","[[play, uno]]","[[game, like, uno]]"
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920,['You should be careful with the symbols (Gold...,"['Thanks for this list!', ""Soooo many games do...","['Thanks for this list!', ""Soooo many games do...","[[symbols], [CARDINAL, rule], [symbols, games,...","[[], [CARDINAL, games, game, rule], [confusing...","[[], [CARDINAL, games, game, rule], [confusing..."
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430,['che bella collezione'],['Che bella collezione 😍😍'],['Che collezione 💪💪💪'],"[[ORG, collezione]]",[[collezione]],[[collezione]]
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100,"['\u200b@@TheHistoryGuyChannel \u200bYah, as I...","[""@@alymbouras it doesn't really matter, the i...","[""@@alymbouras it doesn't really matter, the i...","[[CARDINAL, played, play], [], [WORK_OF_ART, p...","[[], [ORG, games, game, cards]]","[[], [ORG, games, game, cards]]"
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000,['It seems that every country has a card game ...,['One thing most cards are missing (which is w...,['@@StefanLopuszanski I have looked you up and...,"[[game, card], [ORG], [GPE, PERSON, game, play...","[[CARDINAL, playing, cards, card, play], [card...","[[], [PERCENT], [], [], [ORG], [ORG], [ORG], [..."


In [8]:
#transform all the list in a unique one and then eliminate whitespaces.
#filter out cardinals, percent
def extract_unique_words(list_of_lists):
    unique_words = set()
    for sublist in list_of_lists:
        unique_words.update(sublist)
    return [word for word in unique_words if word not in ['CARDINAL', 'PERCENT']]

# Apply the function to the specified columns
for column in ['doc_1_entity_list', 'doc_2_entity_list', 'doc_3_entity_list']:
    df[column] = df[column].apply(extract_unique_words)


df.to_csv(r'/kaggle/working/ent_list.csv', index=False)
df.head(10)

,Name,Labels,Count,Keywords,Topic,docs_1,docs_2,docs_3,docs_1 Sentiment,docs_1 Compound Score,docs_2 Sentiment,docs_2 Compound Score,docs_3 Sentiment,docs_3 Compound Score,docs_1_sentences,docs_2_sentences,docs_3_sentences,doc_1_entity_list,doc_2_entity_list,doc_3_entity_list
0,-1_the_it_to_you,"""Card Game Design and Symbolism""",5706,"['symbols', 'cards', 'text', 'game', 'editing'...",0,This is for everyone Just how you didn’t expec...,#4 is definitely one of the biggest criticisms...,"On adding symbols to the card, I disagree that...",Neutral,-3.244.184.982.031.100,Neutral,-5.191.750.080.455.120,Negative,-8.475.012.043.779.130,['This is for everyone Just how you didn’t exp...,"['#', '4 is definitely one of the biggest crit...","['On adding symbols to the card, I disagree th...","[WORK_OF_ART, TIME, PERSON]","[LOC, cards, ORG, GPE, card, PERSON, NORP, gam...","[ORG, game, card, cards, symbols, text]"
1,Family and Games,Family bonding through Uno,526,"['playing', 'game', 'played', 'games', 'play',...",1,"I'm a game developer, and I realized that my i...",How me and my friends play uno,I like the card game of uno,Positive,8.307.864.991.475.730,Neutral,4.188.930.326.879.080,Positive,13.602.564.403.165.700,"[""I'm a game developer, and I realized that my...",['How me and my friends play uno'],['I like the card game of uno'],"[play, games, watching, PERSON, game, playing]","[uno, play]","[uno, game, like]"
2,Rules of Games,"""Rule breaches in game design""",139,"['symbols', 'rules', 'games', 'confusing', 'pl...",2,You should be careful with the symbols (Golden...,Thanks for this list! Soooo many games don't d...,Thanks for this list! Soooo many games don't d...,Neutral,25.100.642.064.594.400,Negative,-8.003.836.930.573.920,Negative,-8.003.836.930.573.920,['You should be careful with the symbols (Gold...,"['Thanks for this list!', ""Soooo many games do...","['Thanks for this list!', ""Soooo many games do...","[rule, games, game, symbols]","[rule, confusing, games, rules, game, symbols]","[rule, confusing, games, rules, game, symbols]"
3,Games Collection,"""Grande Jack Collection""",123,"['collezione', 'collection', 'bellissima', 'gr...",3,che bella collezione,Che bella collezione 😍😍,Che collezione 💪💪💪,Positive,22.614.071.288.727.800,Positive,23.398.888.646.297.100,Positive,2.243.981.927.813.430,['che bella collezione'],['Che bella collezione 😍😍'],['Che collezione 💪💪💪'],"[collezione, ORG]",[collezione],[collezione]
4,Traditional Games,Card games and techniques,77,"['playing', 'games', 'game', 'played', 'play',...",4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the...","@@alymbouras it doesn't really matter, the imp...","@@alymbouras it doesn't really matter, the imp...",Positive,5.406.065.454.827.610,Neutral,-2.017.972.845.201.100,Neutral,-2.017.972.845.201.100,"['\u200b@@TheHistoryGuyChannel \u200bYah, as I...","[""@@alymbouras it doesn't really matter, the i...","[""@@alymbouras it doesn't really matter, the i...","[WORK_OF_ART, poker, old, played, play, ORG, g...","[cards, games, game, ORG]","[cards, games, game, ORG]"
5,6_the_to_of_and,Card games without indexes,7791,"['playing', 'cards', 'game', 'card', 'games', ...",5,It seems that every country has a card game of...,One thing most cards are missing (which is wei...,@@StefanLopuszanski I have looked you up and y...,Neutral,-56.864.886.756.800.200,Negative,-54.639.819.079.967.200,Negative,-16.614.011.631.226.000,['It seems that every country has a card game ...,['One thing most cards are missing (which is w...,['@@StefanLopuszanski I have looked you up and...,"[LOC, play, ORG, GPE, PERSON, card, cards, game]","[play, games, GPE, card, PERSON, cards, game, ...","[play, ORG, PERSON, card, cards]"


In [21]:
import ast  # To safely evaluate string representations of lists
df = pd.read_csv(r'/kaggle/working/ent_list.csv', encoding="utf-8")

# Convert string representations of lists to actual Python lists
for col in ['doc_1_entity_list', 'doc_2_entity_list', 'doc_3_entity_list']:
    df[col] = df[col].apply(ast.literal_eval)

# Extract entities and matches as previously discussed
def extract_entities(df):
    entity_data = []
    for _, row in df.iterrows():
        topic_id = row['Topic']
        for doc_col in ['doc_1_entity_list', 'doc_2_entity_list', 'doc_3_entity_list']:
            doc_name = doc_col.split('_')[1]  # Extract doc name (e.g., '1', '2', '3')
            for word in row[doc_col]:
                entity_data.append({'word': word, 'topic': topic_id, 'doc': f'{doc_name}_topic_{topic_id}', 'text': row[f'docs_{doc_name}']})
    return pd.DataFrame(entity_data)

# Create a DataFrame with all entities, their topics, and document sources
entities_df = extract_entities(df)

# Perform a self-join to find matches across all rows and columns
matches_df = entities_df.merge(
    entities_df,
    on='word',
    suffixes=('_source', '_target')
)

# Filter out self-matches (where source and target are identical)
matches_df = matches_df[matches_df['doc_source'] != matches_df['doc_target']]

# Eliminate duplicates by ensuring consistent ordering of source and target
matches_df['match'] = matches_df.apply(
    lambda x: tuple(sorted([x['doc_source'], x['doc_target']])), axis=1
)

# Keep only unique matches
uniques = matches_df.drop_duplicates(subset='match').drop(columns='match')

# Save the results to a CSV file (optional)
uniques.to_csv(r'/kaggle/working/matches.csv', index=False)
uniques.head(100)

,word,topic_source,doc_source,text_source,topic_target,doc_target,text_target
1,WORK_OF_ART,0,1_topic_0,This is for everyone Just how you didn’t expec...,4,1_topic_4,"​@@TheHistoryGuyChannel ​Yah, as I recall, the..."
4,PERSON,0,1_topic_0,This is for everyone Just how you didn’t expec...,0,2_topic_0,#4 is definitely one of the biggest criticisms...
5,PERSON,0,1_topic_0,This is for everyone Just how you didn’t expec...,1,1_topic_1,"I'm a game developer, and I realized that my i..."
6,PERSON,0,1_topic_0,This is for everyone Just how you didn’t expec...,5,1_topic_5,It seems that every country has a card game of...
7,PERSON,0,1_topic_0,This is for everyone Just how you didn’t expec...,5,2_topic_5,One thing most cards are missing (which is wei...
...,...,...,...,...,...,...,...
341,cards,4,3_topic_4,"@@alymbouras it doesn't really matter, the imp...",5,2_topic_5,One thing most cards are missing (which is wei...
342,cards,4,3_topic_4,"@@alymbouras it doesn't really matter, the imp...",5,3_topic_5,@@StefanLopuszanski I have looked you up and y...
377,play,5,1_topic_5,It seems that every country has a card game of...,5,2_topic_5,One thing most cards are missing (which is wei...
378,play,5,1_topic_5,It seems that every country has a card game of...,5,3_topic_5,@@StefanLopuszanski I have looked you up and y...


In [13]:
print(uniques)

            word  topic_source doc_source  \
1    WORK_OF_ART             0  1_topic_0   
4         PERSON             0  1_topic_0   
5         PERSON             0  1_topic_0   
6         PERSON             0  1_topic_0   
7         PERSON             0  1_topic_0   
..           ...           ...        ...   
341        cards             4  3_topic_4   
342        cards             4  3_topic_4   
377         play             5  1_topic_5   
378         play             5  1_topic_5   
425         play             5  2_topic_5   

                                           text_source  topic_target  \
1    This is for everyone Just how you didn’t expec...             4   
4    This is for everyone Just how you didn’t expec...             0   
5    This is for everyone Just how you didn’t expec...             1   
6    This is for everyone Just how you didn’t expec...             5   
7    This is for everyone Just how you didn’t expec...             5   
..                         

In [26]:
df = pd.read_csv(r'/kaggle/working/matches.csv', encoding="utf-8")

# Create a graph using networkx
G = nx.Graph()

# Define a color mapping for topics (6 topics)
topic_colors = {
    0: 'red',
    1: 'blue',
    2: 'green',
    3: 'orange',
    4: 'purple',
    5: 'cyan'
}

# Create a Pyvis network visualization
net = Network(notebook=True)

# Add nodes with clustering by document number and coloring by topic
for _, row in uniques.iterrows():
    topic_id_source = int(row['doc_source'].split('_topic_')[1])  # Extract topic ID from source doc name
    topic_id_target = int(row['doc_target'].split('_topic_')[1])  # Extract topic ID from target doc name
    
       # Prepare HTML content for hover frame
    source_text_html = f"{row['text_source']}"
    target_text_html = f"{row['text_target']}"
    
    # Add source node with color based on topic and HTML content as title
    net.add_node(
        row['doc_source'], 
        label=row['doc_source'], 
        color=topic_colors[topic_id_source], 
        title=source_text_html  # Use HTML formatted text for hover
    )
    
    # Add target node with color based on topic and HTML content as title
    net.add_node(
        row['doc_target'], 
        label=row['doc_target'], 
        color=topic_colors[topic_id_target], 
        title=target_text_html  # Use HTML formatted text for hover
    )

# Add edges to the graph from the unique matches DataFrame
for _, row in uniques.iterrows():
    net.add_edge(row['doc_source'], row['doc_target'])

# Convert NetworkX graph to Pyvis format
net.from_nx(G)

net.set_options("""
var options = {
  "nodes": {
    "shape": "dot",
    "size": 10,
    "font": {
      "size": 12,
      "color": "#000000"
    }
  },
  "edges": {
    "color": {
      "highlight": "#ff0000"
    },
    "smooth": false,
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 1,
        "type": "arrow"
      }
    }
  },
  "layout": {
    "hierarchical": false
  }
}
""")

# Save the network visualization to an HTML file
net.save_graph('network_visualization.html')
# Save and show the network visualization
net.show('network_visualization.html')

network_visualization.html
